# About

Convert deeplabv3_mnv2_ade20k_train_2018_12_03 to Core ML

The model can be converted successfully and produces plausible result without modifying any layers. 

Model is quantized to 8 bits. On iPhone Xs, latency reduced from 790 ms (no quantization) to 220 ms.

# Download model and setup enviroments

In [1]:
!rm -rf deeplabv3_mnv2_ade20k_train_2018_12_03
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz
!tar -zxvf deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz

--2021-08-11 17:59:11--  http://download.tensorflow.org/models/deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.119.128, 2a00:1450:4013:c00::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.119.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26028742 (25M) [application/x-tar]
Saving to: ‘deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz’

deeplabv3_mnv2_ade2 100%[===================>]  24.82M  69.2MB/s    in 0.4s    

2021-08-11 17:59:12 (69.2 MB/s) - ‘deeplabv3_mnv2_ade20k_train_2018_12_03.tar.gz’ saved [26028742/26028742]

deeplabv3_mnv2_ade20k_train_2018_12_03/
deeplabv3_mnv2_ade20k_train_2018_12_03/model.ckpt.index
deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb
deeplabv3_mnv2_ade20k_train_2018_12_03/model.ckpt.data-00000-of-00001


In [2]:
%tensorflow_version 1.x

!pip install coremltools==4.1

TensorFlow 1.x selected.
     |████████████████████████████████| 3.4 MB 16.2 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=dd934525cc1d1d33a6ff39d12db39dd92be46b7f4596807f57758c58fec32209
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [3]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
1.15.2


4.1


# Inspect the graph

In [4]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    return graph_def

def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))
  print('Exported to:', filename)


model_name = 'deeplabv3_mnv2_ade20k_train_2018_12_03'
input_file = f'{model_name}/frozen_inference_graph.pb'

with tf.Session(graph=tf.Graph()) as sess:
  graph_def = load_frozen_graph(input_file)
  tf.import_graph_def(graph_def, name="")
  the_graph = tf.compat.v1.get_default_graph()
  export_ops_name(the_graph, f'{model_name}_ops.txt')


Exported to: deeplabv3_mnv2_ade20k_train_2018_12_03_ops.txt


# Convert the TensorFlow model to Core ML.

In [10]:
input_name = "ImageTensor"
output_name = "SemanticPredictions"

input_w = 513
input_h = 513

tfmodel = 'deeplabv3_mnv2_ade20k_train_2018_12_03/frozen_inference_graph.pb'

image_input = ct.ImageType(name=input_name,
                           shape=(1, input_h, input_w, 3),
                           bias=[0,0,0], 
                           scale=1)

coreml_model = ct.convert(
    tfmodel,
    source='tensorflow',
    inputs=[image_input],
    outputs=[output_name]
)

print(coreml_model)

Translating MIL ==> MLModel Ops: 100%|██████████| 877/877 [00:00<00:00, 2290.36 ops/s]


input {
  name: "ImageTensor"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "SemanticPredictions"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==1.15.2"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}



# Quantize the model

In [13]:
from coremltools.models.neural_network import quantization_utils

# allowed values of nbits = 16, 8, 7, 6, ...., 1
spec = quantization_utils.quantize_weights(coreml_model, nbits=8)

Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Fused MobilenetV2/Conv/Conv2Dx->batch_norm_118
Fused MobilenetV2/expanded_conv/depthwise/depthwisex->batch_norm_119
Fused MobilenetV2/expanded_conv/project/Conv2Dx->batch_norm_120
Fused MobilenetV2/expanded_conv_1/expand/Conv2Dx->batch_norm_121
Fused MobilenetV2/expanded_conv_1/depthwise/depthwisex->batch_norm_122
Fused MobilenetV2/expanded_conv_1/project/Conv2Dx->batch_norm_123
Fused MobilenetV2/expanded_conv_2/expand/Conv2Dx->batch_norm_124
Fused MobilenetV2/expanded_conv_2/depthwise/depthwisex->batch_norm_125
Fused MobilenetV2/expanded_conv_2/project/Conv2Dx->batch_norm_126
Fused decoder/feature_projection0/Conv2Dx->batch_norm_127
Fused MobilenetV2/expanded_conv_3/expand/Conv2Dx->batch_norm_128
Fused MobilenetV2/expanded_conv_3/depthwise/depthwisex->batch_norm_129
Fused MobilenetV2/expanded_conv_3/project/Conv2Dx->batch_norm_130
Fused MobilenetV2/expanded_conv_4/expand/Conv2Dx->batch_norm_131
Fused M

In [14]:
# Fill in the descriptions and metadata.
spec.description.metadata.versionString = "2021-08-11"
spec.description.metadata.shortDescription = "deeplabv3_mnv2_ade20k_train_2018_12_03"
spec.description.metadata.author = "Converted by Anh"


# Rename inputs and outputs.
new_input_name = "image"
new_output_name = "scores"

ct.utils.rename_feature(spec, input_name, new_input_name)
ct.utils.rename_feature(spec, output_name, new_output_name)

spec.description.input[0].shortDescription = "Input image"
spec.description.output[0].shortDescription = "Segmentation map"

# Fix up the output shape and make it INT32.
spec.description.output[0].type.multiArrayType.shape.append(input_h)
spec.description.output[0].type.multiArrayType.shape.append(input_w)
spec.description.output[0].type.multiArrayType.dataType = ct.proto.FeatureTypes_pb2.ArrayFeatureType.INT32


print(spec.description)

input {
  name: "image"
  shortDescription: "Input image"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Segmentation map"
  type {
    multiArrayType {
      shape: 513
      shape: 513
      dataType: INT32
    }
  }
}
metadata {
  shortDescription: "deeplabv3_mnv2_ade20k_train_2018_12_03"
  versionString: "2021-08-11"
  author: "Converted by Anh"
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==1.15.2"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}



# Add labels

In [15]:
labels = """
wall
building, edifice
sky
floor, flooring
tree
ceiling
road, route
bed 
windowpane, window 
grass
cabinet
sidewalk, pavement
person, individual, someone, somebody, mortal, soul
earth, ground
door, double door
table
mountain, mount
plant, flora, plant life
curtain, drape, drapery, mantle, pall
chair
car, auto, automobile, machine, motorcar
water
painting, picture
sofa, couch, lounge
shelf
house
sea
mirror
rug, carpet, carpeting
field
armchair
seat
fence, fencing
desk
rock, stone
wardrobe, closet, press
lamp
bathtub, bathing tub, bath, tub
railing, rail
cushion
base, pedestal, stand
box
column, pillar
signboard, sign
chest of drawers, chest, bureau, dresser
counter
sand
sink
skyscraper
fireplace, hearth, open fireplace
refrigerator, icebox
grandstand, covered stand
path
stairs, steps
runway
case, display case, showcase, vitrine
pool table, billiard table, snooker table
pillow
screen door, screen
stairway, staircase
river
bridge, span
bookcase
blind, screen
coffee table, cocktail table
toilet, can, commode, crapper, pot, potty, stool, throne
flower
book
hill
bench
countertop
stove, kitchen stove, range, kitchen range, cooking stove
palm, palm tree
kitchen island
computer, computing machine, computing device, data processor, electronic computer, information processing system
swivel chair
boat
bar
arcade machine
hovel, hut, hutch, shack, shanty
bus, autobus, coach, charabanc, double-decker, jitney, motorbus, motorcoach, omnibus, passenger vehicle
towel
light, light source
truck, motortruck
tower
chandelier, pendant, pendent
awning, sunshade, sunblind
streetlight, street lamp
booth, cubicle, stall, kiosk
television, television receiver, television set, tv, tv set, idiot box, boob tube, telly, goggle box
airplane, aeroplane, plane
dirt track
apparel, wearing apparel, dress, clothes
pole
land, ground, soil
bannister, banister, balustrade, balusters, handrail
escalator, moving staircase, moving stairway
ottoman, pouf, pouffe, puff, hassock
bottle
buffet, counter, sideboard
poster, posting, placard, notice, bill, card
stage
van
ship
fountain
conveyer belt, conveyor belt, conveyer, conveyor, transporter
canopy
washer, automatic washer, washing machine
plaything, toy
swimming pool, swimming bath, natatorium
stool
barrel, cask
basket, handbasket
waterfall, falls
tent, collapsible shelter
bag
minibike, motorbike
cradle
oven
ball
food, solid food
step, stair
tank, storage tank
trade name, brand name, brand, marque
microwave, microwave oven
pot, flowerpot
animal, animate being, beast, brute, creature, fauna
bicycle, bike, wheel, cycle 
lake
dishwasher, dish washer, dishwashing machine
screen, silver screen, projection screen
blanket, cover
sculpture
hood, exhaust hood
sconce
vase
traffic light, traffic signal, stoplight
tray
ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin
fan
pier, wharf, wharfage, dock
crt screen
plate
monitor, monitoring device
bulletin board, notice board
shower
radiator
glass, drinking glass
clock
flag
"""
 
labels_list = labels.split('\n')
print(len(labels_list))
print(labels_list)
assert(len(labels_list) == 152)

152
['', 'wall', 'building, edifice', 'sky', 'floor, flooring', 'tree', 'ceiling', 'road, route', 'bed ', 'windowpane, window ', 'grass', 'cabinet', 'sidewalk, pavement', 'person, individual, someone, somebody, mortal, soul', 'earth, ground', 'door, double door', 'table', 'mountain, mount', 'plant, flora, plant life', 'curtain, drape, drapery, mantle, pall', 'chair', 'car, auto, automobile, machine, motorcar', 'water', 'painting, picture', 'sofa, couch, lounge', 'shelf', 'house', 'sea', 'mirror', 'rug, carpet, carpeting', 'field', 'armchair', 'seat', 'fence, fencing', 'desk', 'rock, stone', 'wardrobe, closet, press', 'lamp', 'bathtub, bathing tub, bath, tub', 'railing, rail', 'cushion', 'base, pedestal, stand', 'box', 'column, pillar', 'signboard, sign', 'chest of drawers, chest, bureau, dresser', 'counter', 'sand', 'sink', 'skyscraper', 'fireplace, hearth, open fireplace', 'refrigerator, icebox', 'grandstand, covered stand', 'path', 'stairs, steps', 'runway', 'case, display case, show

In [16]:
import json
from numpy import loadtxt

labels_json = {"labels": labels_list}

# load the model
mlmodel = ct.models.MLModel(spec)
mlmodel.user_defined_metadata["com.apple.coreml.model.preview.type"] = "imageSegmenter"
mlmodel.user_defined_metadata['com.apple.coreml.model.preview.params'] = json.dumps(labels_json)

mlmodel.save("DeepLabV3_ADE20K.mlmodel")
print(mlmodel)

input {
  name: "image"
  shortDescription: "Input image"
  type {
    imageType {
      width: 513
      height: 513
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Segmentation map"
  type {
    multiArrayType {
      shape: 513
      shape: 513
      dataType: INT32
    }
  }
}
metadata {
  shortDescription: "deeplabv3_mnv2_ade20k_train_2018_12_03"
  versionString: "2021-08-11"
  author: "Converted by Anh"
  userDefined {
    key: "com.apple.coreml.model.preview.params"
    value: "{\"labels\": [\"\", \"wall\", \"building, edifice\", \"sky\", \"floor, flooring\", \"tree\", \"ceiling\", \"road, route\", \"bed \", \"windowpane, window \", \"grass\", \"cabinet\", \"sidewalk, pavement\", \"person, individual, someone, somebody, mortal, soul\", \"earth, ground\", \"door, double door\", \"table\", \"mountain, mount\", \"plant, flora, plant life\", \"curtain, drape, drapery, mantle, pall\", \"chair\", \"car, auto, automobile, machine, motorcar\", \"water\",